# Add a file for tracking

`dvc add test/test_dag.gml`

# Creates a .dvc file

`data.xml.dvc`

# Ignore this in .igitignore

* `git add data/data.xml.dvc data/.gitignore`
* `git commit -m "Add raw data"`

# Upload DVC tracked data

```
dvc remote add -d storage s3://bucket/dvcstore
git add .dvc/config
git commit -m "Configure remote storage"
```

# Push

`dvc push`

# Pull

`dvc pull`

In [ ]:
# default_exp data_handler

In [ ]:
# export

import os
from dataclasses import dataclass
from importlib import import_module
from pathlib import Path
from typing import Any, Dict, List

import numpy as np
import pandas as pd
from sciflow.params import params_as_dict

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# export

supported_parameters = (str, int, float)
supported_args = (Dict, List)
supported_conversion_args = (pd.Series, pd.DataFrame, np.ndarray, Path)
suppported_types = []
suppported_types.extend(supported_parameters)
suppported_types.extend(supported_args)
suppported_types.extend(supported_conversion_args)

In [ ]:
# export


@dataclass
class ParamMeta:
    instance_type: type
    is_scalar: bool
    has_metaflow_param: bool
    is_json_type: bool
    persist_type: str

In [ ]:
# export


def load_module(fully_qualified_module_name):
    package = ".".join(fully_qualified_module_name.split(".")[:-1])
    fully_qualified_module_name.split(".")[-1]
    module = import_module(fully_qualified_module_name, package)
    return module

In [ ]:
assert load_module("sciflow.test.test_data_handling").int_param == 3
assert load_module("sciflow.test.test_data_handling").float_param == 1.1
assert type(load_module("sciflow.test.test_data_handling").series_param) == pd.Series
assert load_module("sciflow.utils").load_dremio_access is not None

In [ ]:
# export


def extract_param_meta(module_name: str, params: Dict[str, Any]):
    module = load_module(module_name)
    param_vals = {}
    for k in params.keys():
        param_vals[k] = getattr(module, k)
    param_meta = {}

    for key, val in param_vals.items():
        if any([isinstance(val, t) for t in supported_parameters]):
            param_meta[key] = ParamMeta(
                instance_type=type(val),
                is_scalar=True,
                has_metaflow_param=True,
                is_json_type=False,
                persist_type="pickle",
            )
        elif any([isinstance(val, t) for t in supported_args]):
            param_meta[key] = ParamMeta(
                instance_type=type(val),
                is_scalar=False,
                has_metaflow_param=True,
                is_json_type=True,
                persist_type="pickle",
            )
        elif any([isinstance(val, t) for t in supported_conversion_args]):
            if isinstance(val, np.ndarray):
                param_meta[key] = ParamMeta(
                    instance_type=type(val),
                    is_scalar=False,
                    has_metaflow_param=False,
                    is_json_type=False,
                    persist_type="numpy",
                )
            elif isinstance(val, pd.Series) or isinstance(val, pd.DataFrame):
                param_meta[key] = ParamMeta(
                    instance_type=type(val),
                    is_scalar=False,
                    has_metaflow_param=False,
                    is_json_type=False,
                    persist_type="pandas",
                )
            elif isinstance(val, Path):
                param_meta[key] = ParamMeta(
                    instance_type=type(val),
                    is_scalar=False,
                    has_metaflow_param=True,
                    is_json_type=False,
                    persist_type="pickle",
                )
        else:
            param_meta[key] = ParamMeta(
                instance_type=type(val),
                is_scalar=False,
                has_metaflow_param=False,
                is_json_type=False,
                persist_type="unsupported",
            )
    return param_meta

In [ ]:
nb_path = os.path.join(Path(".").resolve(), "test", "test_data_handling.ipynb")
params = params_as_dict(nb_path)

In [ ]:
param_meta = extract_param_meta("sciflow.test.test_data_handling", params)

In [ ]:
from pathlib import PosixPath

In [ ]:
assert all(
    [
        param_meta["int_param"].instance_type == int,
        param_meta["float_param"].instance_type == float,
        param_meta["str_param"].instance_type == str,
        param_meta["input_path"].instance_type == PosixPath,
        param_meta["model_path"].instance_type == PosixPath,
        param_meta["dict_param"].instance_type == dict,
        param_meta["list_param"].instance_type == list,
        param_meta["series_param"].instance_type == pd.Series,
        param_meta["df_param"].instance_type == pd.DataFrame,
    ]
)

In [ ]:
nb_path = os.path.join(Path(".").resolve(), "test", "test_clustering.ipynb")
params = params_as_dict(nb_path)
param_meta = extract_param_meta("sciflow.test.test_clustering", params)

In [ ]:
param_meta.keys()

dict_keys(['traffic_percent', 'speed', 'workers', 'dremio_access', 'model_level', 'min_date'])

In [ ]:
param_meta["dremio_access"]

ParamMeta(instance_type=<class 'data_access.dremio_access.DremioAccess'>, is_scalar=False, has_metaflow_param=False, is_json_type=False, persist_type='unsupported')